In [6]:
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from profanity import profanity

stopwords = stopwords.words('english')

df = pd.read_json('jokes/wocka.json')
del df['id']

df.head()

,body,category,title
0,What do you call a cow with no legs?\r\n\r\nGr...,Animal,Cow With No Legs
1,What do you call a cow jumping over a barbed w...,Animal,Jumping Cow
2,What's black and white and red all over?\r\n\r...,Other / Misc,"Black, White and Red"
3,"So, this guy walks into a bar.\r\n\r\nAnd says...",Bar,Guy in a Bar
4,"If the opposite of pro is con, isn't the oppos...",One Liners,Progress


In [7]:
tokenizer = RegexpTokenizer(r'\w+')
p_stemmer = PorterStemmer()

tokenized_jokes = pd.Series(index=df.index)

to_drop = []
for index, joke in df.iterrows():
    full_text = joke.title + ' ' + joke.body
    full_text = tokenizer.tokenize(full_text)
    if any(word in profanity for word in full_text):
        to_drop.append(index)
    full_text = [p_stemmer.stem(word) for word in full_text 
                 if word not in stopwords]
    
    tokenized_jokes[index] = ' '.join(full_text).upper()

token_df = df.assign(tokenized_joke=tokenized_jokes)
    
print(token_df.shape[0])
token_df = token_df.drop(to_drop)
print(token_df.shape[0])

10019
9064


In [8]:
import pickle

with open('tokenized_joke.p', 'wb') as fp:
    pickle.dump(token_df, fp)

token_df.head()

,body,category,title,tokenized_joke
0,What do you call a cow with no legs?\r\n\r\nGr...,Animal,Cow With No Legs,COW WITH NO LEG WHAT CALL COW LEG GROUND BEEF
1,What do you call a cow jumping over a barbed w...,Animal,Jumping Cow,JUMP COW WHAT CALL COW JUMP BARB WIRE FENC UTT...
2,What's black and white and red all over?\r\n\r...,Other / Misc,"Black, White and Red",BLACK WHITE RED WHAT BLACK WHITE RED A NEWSPAP
3,"So, this guy walks into a bar.\r\n\r\nAnd says...",Bar,Guy in a Bar,GUY BAR SO GUY WALK BAR AND SAY OUCH
4,"If the opposite of pro is con, isn't the oppos...",One Liners,Progress,PROGRESS IF OPPOSIT PRO CON OPPOSIT PROGRESS C...
